In [1]:
import numpy as np
import cv2 
import random
import math
import os
import datetime
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, Concatenate
from sklearn.utils.class_weight import compute_class_weight
%load_ext tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# PATH = ''
PATH = '/content/drive/MyDrive/NFSMW/noshift/'


pickle_in = open(PATH + 'x_speed.pickle', 'rb')
x_speed = pickle.load(pickle_in)
pickle_in.close()

x_speed = x_speed / 1000

# pickle_in = open(PATH + 'y_acceleration.pickle', 'rb')
# y_acceleration = pickle.load(pickle_in)
# pickle_in.close()
pickle_in = open(PATH + 'y_direction.pickle', 'rb')
y_direction = pickle.load(pickle_in)
pickle_in.close()


INPUT_SHAPE = np.array([135, 240, 1])
NUM_SAMPLES = x_speed.shape[0]

In [4]:
input_speed = Input(shape=(1,), name='speed')
input_img = Input(shape=INPUT_SHAPE, name='img')


# ALEXNET img
x = Conv2D(96, kernel_size=(11,11), strides= 4,
            padding= 'valid', activation= 'relu',
            # input_shape=x_img[1:].shape,
            kernel_initializer= 'he_normal')(input_img)
x = MaxPooling2D(pool_size=(3,3), strides= (2,2),
                    padding= 'valid', data_format= None)(x)
x = BatchNormalization()(x)

x = Conv2D(256, kernel_size=(5,5), strides= 1,
            padding= 'same', activation= 'relu',
            kernel_initializer= 'he_normal')(x)
x = MaxPooling2D(pool_size=(3,3), strides= (2,2),
                    padding= 'valid', data_format= None)(x) 
x = BatchNormalization()(x)

x = Conv2D(384, kernel_size=(3,3), strides= 1,
            padding= 'same', activation= 'relu',
            kernel_initializer= 'he_normal')(x)
x = Conv2D(384, kernel_size=(3,3), strides= 1,
            padding= 'same', activation= 'relu',
            kernel_initializer= 'he_normal')(x)
x = Conv2D(256, kernel_size=(3,3), strides= 1,
            padding= 'same', activation= 'relu',
            kernel_initializer= 'he_normal')(x)
x = MaxPooling2D(pool_size=(3,3), strides= (2,2),
                    padding= 'valid', data_format= None)(x)
x = Conv2D(256, kernel_size=(5,5), strides= 1,
            padding= 'same', activation= 'relu',
            kernel_initializer= 'he_normal')(x)
x = MaxPooling2D(pool_size=(3,3), strides= (2,2),
                    padding= 'valid', data_format= None)(x) 
x = BatchNormalization()(x)

x = Conv2D(384, kernel_size=(3,3), strides= 1,
            padding= 'same', activation= 'relu',
            kernel_initializer= 'he_normal')(x)
x = Conv2D(384, kernel_size=(3,3), strides= 1,
            padding= 'same', activation= 'relu',
            kernel_initializer= 'he_normal')(x)
x = Conv2D(256, kernel_size=(3,3), strides= 1,
            padding= 'same', activation= 'relu',
            kernel_initializer= 'he_normal')(x)
x = MaxPooling2D(pool_size=(3,3), strides= (2,2),
                    padding= 'same', data_format= None)(x)
x = BatchNormalization()(x)

x = Flatten()(x)

# add speed input
x = Concatenate(axis=1)([x, input_speed])

SIZE_DENSE = 1024
# dense for outputs
x_direction = Dense(SIZE_DENSE, activation= 'sigmoid')(x)
x_direction = Dropout(0.5)(x_direction)
x_direction = Dense(SIZE_DENSE, activation= 'sigmoid')(x_direction)
x_direction = Dropout(0.5)(x_direction)
x_direction = Dense(SIZE_DENSE, activation= 'sigmoid')(x_direction)
x_direction = Dropout(0.5)(x_direction)
x_direction = Dense(3, activation= 'softmax', name='direction')(x_direction)

# x_acceleration = Dense(SIZE_DENSE, activation= 'tanh')(x)
# x_acceleration = Dropout(0.5)(x_acceleration)
# x_acceleration = Dense(SIZE_DENSE, activation= 'tanh')(x_acceleration)
# x_acceleration = Dropout(0.5)(x_acceleration)
# x_acceleration = Dense(SIZE_DENSE/4, activation= 'tanh')(x_acceleration)
# x_acceleration = Dropout(0.5)(x_acceleration)
# x_acceleration = Dense(3, activation= 'softmax', name='acceleration')(x_acceleration)


# [FAILED ATTEMPT TO MULTIPLE OUTPUTS CLASS WEIGHTS]
# def class_loss(class_weight):
#   """Returns a loss function for a specific class weight tensor
  
#   Params:
#     class_weight: 1-D constant tensor of class weights
    
#   Returns:
#     A loss function where each loss is scaled according to the observed class"""
#   def loss(y_obs, y_pred):
#     y_obs = tf.dtypes.cast(y_obs, tf.int32)
#     hothot = tf.one_hot(tf.reshape(y_obs, [-1]), depth=class_weight.shape[0])
#     weight = tf.math.multiply(class_weight, hothot)
#     weight = tf.reduce_sum(weight, axis=-1)
#     losses = tf.compat.v1.losses.sparse_softmax_cross_entropy(labels=y_obs,
#                                                               logits=y_pred,
#                                                               weights=weight)
#     return losses
#   return loss

# MULTIPLE OUTPUTS
# model = Model(inputs=[input_img, input_speed], 
#               outputs = [x_direction, x_acceleration])

# model.compile(optimizer= tf.keras.optimizers.Adam(0.001),
#         loss=['categorical_crossentropy', 'categorical_crossentropy'],
#         metrics=['accuracy'])

# SINGLE OUTPUT
model_direction = Model(inputs=[input_img, input_speed], 
              outputs=x_direction)

model_direction.compile(optimizer= tf.keras.optimizers.Adam(0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy'])

# model_acceleration = Model(inputs=[input_img, input_speed], 
#               outputs=x_direction)

# model_acceleration.compile(optimizer= tf.keras.optimizers.Adam(0.001),
#         loss='categorical_crossentropy',
#         metrics=['accuracy'])

# model.fit({'img': x_img, 'speed': x_speed}, y_direction, epochs=30, validation_split=0.15, 
          # class_weight=class_weights['direction'],
#           # callbacks=[tensorboard_callback]
#           )


# model.compile(optimizer= tf.keras.optimizers.Adam(0.001),
#         loss={k: class_loss(v) for k, v in class_weights.items()},
#         metrics=['accuracy'])

In [5]:
# %tensorboard --logdir logs

In [6]:
model_direction = tf.keras.models.load_model('/content/drive/MyDrive/NFSMW/alex_dir_100k_4epoch.model')

In [7]:
# model_direction.summary()

In [8]:
EPOCHS = 1

for epoch in range(EPOCHS):
  print(f'\n================ Epoch {epoch+1} ================\n')
  for name in os.listdir(PATH + 'x_img'):
      data =  name.split('_')
      start = int(data[0])
      finish = int(data[1])

      pickle_in = open(PATH + f'x_img/{name}', 'rb')
      x_img = pickle.load(pickle_in)
      pickle_in.close()
      
      x_img = x_img / 255.0

      class_weights = compute_class_weight(y=np.array([np.argmax(arr) for arr in y_direction[start:finish]]), class_weight='balanced', classes=[0, 1, 2])

      model_direction.fit({'img': x_img, 'speed': x_speed[start:finish]}, y_direction[start:finish], epochs=5, validation_split=0.1,
                class_weight={i: weight for i, weight in enumerate(class_weights)},
                callbacks =[tensorboard_callback]
                )
      
      del x_img


================ Epoch 1 ================

Epoch 1/5
141/141 [==============================] - 15s 61ms/step - loss: 0.7268 - accuracy: 0.6889 - val_loss: 0.7504 - val_accuracy: 0.6820
Epoch 2/5
141/141 [==============================] - 7s 48ms/step - loss: 0.5445 - accuracy: 0.7638 - val_loss: 0.8774 - val_accuracy: 0.6640
Epoch 3/5
141/141 [==============================] - 8s 54ms/step - loss: 0.4110 - accuracy: 0.8460 - val_loss: 0.9250 - val_accuracy: 0.6500
Epoch 4/5
141/141 [==============================] - 7s 46ms/step - loss: 0.2805 - accuracy: 0.9047 - val_loss: 1.0655 - val_accuracy: 0.6580
Epoch 5/5
141/141 [==============================] - 6s 44ms/step - loss: 0.2300 - accuracy: 0.9300 - val_loss: 1.0076 - val_accuracy: 0.6920
Epoch 1/5
141/141 [==============================] - 7s 51ms/step - loss: 0.7222 - accuracy: 0.6833 - val_loss: 0.7709 - val_accuracy: 0.6060
Epoch 2/5
141/141 [==============================] - 6s 45ms/step - loss: 0.5697 - accuracy: 0.7700 - v

In [9]:
model_direction.save('/content/drive/MyDrive/NFSMW/alex_dir_100k_5epoch.model')

In [ ]:
model_acceleration.fit({'img': x_img, 'speed': x_speed}, y_acceleration, epochs=30, validation_split=0.15, 
          class_weight=class_weights['acceleration'], 
          # callbacks=[tensorboard_callback]
          )

In [ ]:
# model = Sequential()

# model.add(Conv2D(256, (3, 3), input_shape=x_img.shape[1:]))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(256, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(256, (3, 3)))
# model.add(Activation('relu'))
# model.add(Conv2D(256, (3, 3)))
# model.add(Activation('relu'))
# model.add(Conv2D(256, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dense(256))
# model.add(Activation('relu'))

# model.add(Dense(3))
# model.add(Activation('softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], lr=0.001)

# model.fit(x_img, y_direction, epochs=15, validation_split=0.1, class_weight={i: weight for i, weight in enumerate(class_weights_direction)})

In [ ]:
# model.fit({'img': x_img, 'speed': x_speed}, {'acceleration': y_acceleration, 'direction': y_direction}, epochs=3, validation_split=0.15)

In [ ]:
# model_direction.save('/content/drive/MyDrive/ML/NFSMW/alex_dir.model')
# model_acceleration.save('/content/drive/MyDrive/ML/NFSMW/alex_acc.model')

# from google.colab import files
# !zip -r /content/model.zip simple_nofilter.model

In [ ]:
# # AlexNet model
# class AlexNet(Sequential):
#     def __init__(self, input_shape, num_classes):
#         super().__init__()

#         self.add(Conv2D(96, kernel_size=(11,11), strides= 4,
#                         padding= 'valid', activation= 'relu',
#                         input_shape= input_shape,
#                         kernel_initializer= 'he_normal'))
#         self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
#                               padding= 'valid', data_format= None))

#         self.add(Conv2D(256, kernel_size=(5,5), strides= 1,
#                         padding= 'same', activation= 'relu',
#                         kernel_initializer= 'he_normal'))
#         self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
#                               padding= 'valid', data_format= None)) 

#         self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
#                         padding= 'same', activation= 'relu',
#                         kernel_initializer= 'he_normal'))

#         self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
#                         padding= 'same', activation= 'relu',
#                         kernel_initializer= 'he_normal'))

#         self.add(Conv2D(256, kernel_size=(3,3), strides= 1,
#                         padding= 'same', activation= 'relu',
#                         kernel_initializer= 'he_normal'))

#         self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
#                               padding= 'valid', data_format= None))

#         self.add(Flatten())
#         self.add(Dense(4096, activation= 'relu'))
#         self.add(Dense(4096, activation= 'relu'))
#         self.add(Dense(1000, activation= 'relu'))
#         self.add(Dense(num_classes, activation= 'softmax'))

#         self.compile(optimizer= tf.keras.optimizers.Adam(0.001),
#                     loss='categorical_crossentropy',
#                     metrics=['accuracy'])


# def alexnet(width, height, lr):
#     network = input_data(shape=[None, width, height, 1], name='input')
#     network = conv_2d(network, 96, 11, strides=4, activation='relu')
#     network = max_pool_2d(network, 3, strides=2)
#     network = local_response_normalization(network)
#     network = conv_2d(network, 256, 5, activation='relu')
#     network = max_pool_2d(network, 3, strides=2)
#     network = local_response_normalization(network)
#     network = conv_2d(network, 384, 3, activation='relu')
#     network = conv_2d(network, 384, 3, activation='relu')
#     network = conv_2d(network, 256, 3, activation='relu')
#     network = max_pool_2d(network, 3, strides=2)
#     network = local_response_normalization(network)
#     network = fully_connected(network, 4096, activation='tanh')
#     network = dropout(network, 0.5)
#     network = fully_connected(network, 4096, activation='tanh')
#     network = dropout(network, 0.5)
#     network = fully_connected(network, 3, activation='softmax')
#     network = regression(network, optimizer='momentum',
#                          loss='categorical_crossentropy',
#                          learning_rate=lr, name='targets')

#     model = tflearn.DNN(network, checkpoint_path='model_alexnet',
#                         max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir='log')

#     return model

In [ ]:
# # AlexNet model
# class AlexNet(Sequential):
#     def __init__(self, input_shape, num_classes):
#         super().__init__()

#         self.add(Conv2D(96, kernel_size=(11,11), strides= 4,
#                         padding= 'valid', activation= 'relu',
#                         input_shape= input_shape,
#                         kernel_initializer= 'he_normal'))
#         self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
#                               padding= 'valid', data_format= None))

#         self.add(Conv2D(256, kernel_size=(5,5), strides= 1,
#                         padding= 'same', activation= 'relu',
#                         kernel_initializer= 'he_normal'))
#         self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
#                               padding= 'valid', data_format= None)) 

#         self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
#                         padding= 'same', activation= 'relu',
#                         kernel_initializer= 'he_normal'))

#         self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
#                         padding= 'same', activation= 'relu',
#                         kernel_initializer= 'he_normal'))

#         self.add(Conv2D(256, kernel_size=(3,3), strides= 1,
#                         padding= 'same', activation= 'relu',
#                         kernel_initializer= 'he_normal'))

#         self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
#                               padding= 'valid', data_format= None))

#         self.add(Flatten())
#         self.add(Dense(4096, activation= 'relu'))
#         self.add(Dense(4096, activation= 'relu'))
#         self.add(Dense(1000, activation= 'relu'))
#         self.add(Dense(num_classes, activation= 'softmax'))

#         self.compile(optimizer= tf.keras.optimizers.Adam(0.001),
#                     loss='categorical_crossentropy',
#                     metrics=['accuracy'])
        
# model_direction = AlexNet(input_shape=x_img.shape[1:], num_classes=3)

In [ ]:
import gc

x_img = get_imgs(0, 5000)
x_img = x_img / 255.0

for _ in range(30):
  idx = np.random.randint(0, 5000)


  tf.keras.backend.clear_session()
  gc.collect()
  output = model_direction({'img': x_img[idx].reshape(1, 135, 240, 1), 'speed': np.array([x_speed[idx]])})
  tf.keras.backend.clear_session()
  gc.collect()

  # direction, acceleration = output[0].numpy()[0], output[1].numpy()[0]

  plt.figure()
  plt.imshow(x_img[idx, :, :, 0], cmap='gray')
  plt.xticks([]), plt.yticks([])
  plt.title(f'{output.numpy()[0]}; {y_direction[idx]}')
  plt.show()

NameError: ignored